<a href="https://colab.research.google.com/github/detektor777/colab_list_image/blob/main/sdxl_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ##**Install** { display-mode: "form" }
%%capture
!pip install -q invisible_watermark transformers accelerate safetensors diffusers controlnet_aux==0.0.7 xformers mediapy
import torch
from diffusers import StableDiffusionXLImg2ImgPipeline, DiffusionPipeline, KDPM2AncestralDiscreteScheduler, StableDiffusionXLPipeline, AutoencoderKL
import gc
from PIL import Image
import requests
from io import BytesIO
from IPython.display import display
model_base = "stabilityai/stable-diffusion-xl-base-1.0"
v_autoencoder = "madebyollin/sdxl-vae-fp16-fix" # fix vae for run in fp16 precision without generating NaNs

vae = AutoencoderKL.from_pretrained(v_autoencoder, torch_dtype=torch.float16)

pipe = StableDiffusionXLPipeline.from_pretrained(
    model_base,
    vae=vae,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    add_watermarker=False, # no watermarker
    )

pipe.safety_checker = None

pipe.to("cuda")
model_refiner = "stabilityai/stable-diffusion-xl-refiner-1.0"

pipe_refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    model_refiner,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    add_watermarker=False, # no watermarker
    )

#pipe_refiner.to("cuda")
pipe_refiner.enable_model_cpu_offload()
#(Optional) Change the scheduler
pipe.scheduler = KDPM2AncestralDiscreteScheduler.from_config(
pipe.scheduler.config, use_karras_sigmas=False
)
#generator = torch.Generator().manual_seed(42)

In [ ]:
#@title ##**Create** { display-mode: "form" }
prompt =  "Ukrainian flag by Pascal Campion, Luis Royo" #@param {type:"string"}
prompt2 = ""


negative_prompt = ''#@param {type:"string"}
negative_prompt2 = ''

scale = 10 #@param {type:"integer"}
steps = 30 #@param {type:"integer"}

height = 1024 #@param {type:"integer"}
width = 1024 #@param {type:"integer"}

image_base = pipe(
    prompt=prompt,
    prompt_2=prompt2,
    height=1024,
    width=1024,
    negative_prompt=negative_prompt,
    negative_prompt_2=negative_prompt2,
    guidance_scale=scale,
    num_inference_steps=steps,
    ).images[0]

gc.collect()
torch.cuda.empty_cache()

In [ ]:
#@title ##**Show** { display-mode: "form" }
image_base